In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
from cifar_model import resnet50
from tqdm import tqdm
import copy
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

In [3]:
torch.__version__, torchvision.__version__

('1.4.0', '0.5.0')

In [4]:
device = 'cuda:0'

In [5]:
def train_val(model, optimizer, train_loader, val_loader, epochs=10):
    model.train()
    val_accs = []
    best_val_loss = np.inf
    patience_counter = 0
    for epoch in range(epochs):
        # Train
        train_loss, train_preds, train_labels = [], [], []
        model.train()
        pbar = tqdm(train_loader, position=0, leave=True)
        for i, (data, label) in enumerate(pbar):                                            
            data, label = data.to(device), label.to(device)
            optimizer.zero_grad()
            logits = model(data)
            loss = F.cross_entropy(logits, label)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(logits, dim=1)
            acc = (preds == label).type(torch.FloatTensor).mean()
            train_loss.append(loss.item()), train_preds.append(preds), train_labels.append(label)
            train_loss_mean = torch.Tensor(train_loss).mean()
            train_acc_mean = (torch.cat(train_preds) == 
                              torch.cat(train_labels)).type(torch.FloatTensor).mean().item()
            pbar.set_postfix({'epoch': epoch, 
                              'loss': f'{train_loss_mean:.2f}', 
                              'acc': f'{train_acc_mean:.2f}'})
        # Validation
        model.eval()
        val_loss, val_preds, val_labels = [], [], []
        pbar = tqdm(val_loader, position=0, leave=True)
        for i, (data, label) in enumerate(pbar):
            data, label = data.to(device), label.to(device)
            with torch.no_grad():
                logits = model(data)
                loss = F.cross_entropy(logits, label).item()
                preds = torch.argmax(logits, dim=1)
                acc = (preds == label).type(torch.FloatTensor).mean().item()
                val_loss.append(loss), val_preds.append(preds), val_labels.append(label)
            val_loss_mean = torch.Tensor(val_loss).mean()
            val_acc_mean = (torch.cat(val_preds) == 
                            torch.cat(val_labels)).type(torch.FloatTensor).mean().item()
            pbar.set_postfix({'epoch': epoch, 
                              'val loss': f'{val_loss_mean:.2f}', 
                              'val acc': f'{val_acc_mean:.2f}'})

In [11]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
val_loader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
model = resnet50(num_classes=len(trainset.classes))
model = torch.nn.DataParallel(model)
model.to(device);

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=20)

100%|██████████| 100/100 [00:07<00:00, 13.58it/s, epoch=19, val loss=0.49, val acc=0.84]


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=2)

100%|██████████| 100/100 [00:07<00:00, 13.12it/s, epoch=1, val loss=0.25, val acc=0.92]


In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-4)
train_val(model, optimizer, train_loader, val_loader, epochs=10)

100%|██████████| 100/100 [00:07<00:00, 13.08it/s, epoch=9, val loss=0.25, val acc=0.93]
